In [20]:
 # activate Spark in our Colab notebook.
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example: 'spark-3.0.2'
spark_version = 'spark-3.1.1'
# spark_version = 'spark-3.<enter version>'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:13 http://ppa.launchpad

In [21]:
 #import packages

from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql.types import StructType,StructField,StringType, DateType,IntegerType
from pyspark import SparkFiles

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [22]:
#reading the peterpan text file
df = spark.read.option("header", "true") \
    .option("delimiter", ",") \
    .option("inferSchema", "true") \
    .csv("/content/peterpan.txt")

In [33]:
#taking a look at the file
#df.show()


In [34]:
#importing libraries 
from pyspark import SparkConf
from pyspark.context import SparkContext
sc = SparkContext.getOrCreate(SparkConf())
data = sc.textFile("/content/peterpan.txt")

In [35]:
#using lamda to count
counts = data.flatMap(lambda line: line.split(" ")) \
             .map(lambda word: (word, 1)) \
             .reduceByKey(lambda a, b: a + b)

In [37]:
#saving ouput of count as text file
counts.saveAsTextFile("/content/peterpanBookWORDcounts")

In [38]:
#reading into df_1 so to print out
df_1 = spark.read.option("header", "true") \
    .option("delimiter", ",") \
    .option("inferSchema", "true") \
    .csv("/content/peterpanBookWORDcounts")

In [39]:
df_1.show()

+------------+-----+----+
|       ('one|    '| 15)|
+------------+-----+----+
|     ('They'| 101)|null|
|     ('know'|  64)|null|
|      ('way'|  58)|null|
|      ('was'| 897)|null|
|      ('One'|   9)|null|
|     ('when'| 151)|null|
|      ('two'|  36)|null|
|    ('years'|   3)|null|
|       ('in'| 623)|null|
|    ('garden|    '|  1)|
|  ('plucked'|   2)|null|
|   ('flower'|   2)|null|
|      ('ran'|  17)|null|
|      ('her'| 361)|null|
|  ('suppose'|   7)|null|
|     ('must'|  59)|null|
|     ('have'| 243)|null|
|   ('looked'|  33)|null|
|   ('rather'|  40)|null|
|('delightful|    '|  1)|
+------------+-----+----+
only showing top 20 rows

